In [67]:
import arcpy
from arcpy.sa import *
import os
import time
arcpy.env.overwriteOutput = True

In [68]:
# load input features and make layer
output_dir = r'J:\lakemapping\postprocess\v4_2403019'
auxiliary_dataset_gdb=r'D:\postprocess\v240220\auxiliary_dataset.gdb'
river_mask_gdb=os.path.join(output_dir,f'2_polygon_iw_River.gdb')
after_mask_gdb=os.path.join(output_dir,f'3_polygon_after_rivermask.gdb')
merge_gdb=os.path.join(output_dir, "4_polygon_merge.gdb")
region="as_4"
raw_prediction_fn='lakes_'+region
prefix=region
arcpy.env.workspace =merge_gdb 

In [3]:
GLAKES_Res=os.path.join(auxiliary_dataset_gdb,'GLAKES_Res')
arcpy.MakeFeatureLayer_management(GLAKES_Res,'GLAKES_Res')

GLAKES=os.path.join(auxiliary_dataset_gdb,r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES\GLAKES.gdb\GLAKES')
arcpy.MakeFeatureLayer_management(GLAKES,'GLAKES')

osm_reservoir=os.path.join(auxiliary_dataset_gdb,'osm_as_natural_water_reservoir_dam')
arcpy.MakeFeatureLayer_management(osm_reservoir,'osm_reservoir')

GeoDAR=r'D:\lakemapping\0_auxiliary_data\GeoDAR\GeoDAR_v10_v11\GeoDAR_v11_reservoirs.shp'
arcpy.MakeFeatureLayer_management(GeoDAR,'GeoDAR')


<Result 'GeoDAR'>

In [16]:
lake_arm=os.path.join(river_mask_gdb,raw_prediction_fn+'_arm')
lake_iwG=os.path.join(river_mask_gdb,raw_prediction_fn+'_iwG')
lake_niwG=os.path.join(river_mask_gdb,raw_prediction_fn+'_niwG')
lake_iwG_arm_SJ=lake_iwG+'_arm_SJ'
lake_niwG_arm_SJ=lake_niwG+'_arm_SJ'
lake_waterway_niwG=os.path.join(river_mask_gdb,raw_prediction_fn+'_waterway_niwG')
lake_waterway_iwG_SJ=os.path.join(river_mask_gdb,raw_prediction_fn+'_waterway_iwG_SJ')
lake_niwR=os.path.join(river_mask_gdb,raw_prediction_fn+'_niwR')
big_lake=r'J:\lakemapping\postprocess\V240303\1_other_polygons.gdb\lakes_{}_gte02'.format(region)
lake_merge_bl=os.path.join(merge_gdb,raw_prediction_fn+'_merge_biglake')
GLAKES_iwRser=os.path.join(merge_gdb,f'GLAKES_iw_{region}_Rser_2')
GLAKES_iwRser_clip=os.path.join(merge_gdb,f'GLAKES_iw_{region}_Rser_clip')
GLAKES_iwRser_join=os.path.join(merge_gdb,f'GLAKES_iw_{region}_Rser_join')

In [17]:
GLAKES_lyr='GLAKES'
lake_arm_lyr=f'lakes_{region}_arm'
lake_niwR_lyr=f'lakes_{region}_niwR'
lake_iwG_arm_SJ_lyr=f'lakes_{region}_iwG_arm_SJ'
lake_niwG_arm_SJ_lyr=f'lakes_{region}_niwG_arm_SJ'
lake_waterway_niwG_lyr=f'lakes_{region}_waterway_niwG'
lake_waterway_iwG_SJ_lyr=f'lakes_{region}_waterway_iwG_SJ'
lake_big_lake_lyr=f'lakes_{region}_biglakeModel'
lake_merge_bl_lyr=f'lake_{region}_merge_bl'
# GLAKES_iwRser_lyr=f'GLAKES_iw_{region}_Rser'
# GLAKES_iwRser_clip_lyr=f'GLAKES_iw_{region}_clip_Rser'
# GLAKES_iwRser_join_lyr=f'GLAKES_iw_{region}_Rser_join'

In [22]:
arcpy.MakeFeatureLayer_management(GLAKES,GLAKES_lyr)
arcpy.MakeFeatureLayer_management(lake_arm,lake_arm_lyr)
arcpy.MakeFeatureLayer_management(lake_niwR,lake_niwR_lyr)
arcpy.MakeFeatureLayer_management(lake_iwG_arm_SJ,lake_iwG_arm_SJ_lyr)
arcpy.MakeFeatureLayer_management(lake_niwG_arm_SJ,lake_niwG_arm_SJ_lyr)
arcpy.MakeFeatureLayer_management(lake_waterway_niwG,lake_waterway_niwG_lyr)
arcpy.MakeFeatureLayer_management(lake_waterway_iwG_SJ,lake_waterway_iwG_SJ_lyr)
arcpy.MakeFeatureLayer_management(big_lake,lake_big_lake_lyr)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset J:\lakemapping\postprocess\V240303\1_other_polygons.gdb\lakes_as_4_gte02 does not exist or is not supported
Failed to execute (MakeFeatureLayer).


In [18]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateField(inputFeature,field,code):
    arcpy.CalculateField_management(inputFeature, field,code)

In [19]:
temp_file=lake_merge_bl+'_temp'
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(lake_iwG_arm_SJ_lyr)#未mask
field_list = ['area','flag','operate']  # keep the only three fields, area代表lake without rivermask的面积，area_arm代表lake with rivermask的面积
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))

selectByAttribute(lake_iwG_arm_SJ_lyr,'NEW_SELECTION','"operate" = 0.5 OR operate IS NULL')
selectByLocation(lake_arm_lyr, 'WITHIN', lake_iwG_arm_SJ_lyr)

selectByAttribute(lake_niwG_arm_SJ_lyr,'NEW_SELECTION','"operate" = 0.5 OR operate IS NULL')
selectByLocation(lake_arm_lyr, 'WITHIN', lake_niwG_arm_SJ_lyr,'ADD_TO_SELECTION')

selectByAttribute(lake_arm_lyr,'SUBSET_SELECTION','"operate" IS NULL')
selectByAttribute(lake_iwG_arm_SJ_lyr,'NEW_SELECTION','"operate" >= 1')
selectByAttribute(lake_niwG_arm_SJ_lyr,'NEW_SELECTION','"operate" >= 1')

selectByAttribute(lake_waterway_iwG_SJ_lyr,'NEW_SELECTION','"operate"<> 0 or operate IS NULL')
selectByAttribute(lake_waterway_niwG_lyr,'NEW_SELECTION','"operate"<> 0 or operate IS NULL')

selectByAttribute(lake_niwR_lyr,'NEW_SELECTION','"operate"<> 0 or operate IS NULL')
# selectByAttribute(big_lake_merge,'NEW_SELECTION','"operate"=1')
selectByAttribute(lake_big_lake_lyr,'NEW_SELECTION','"operate"=1')
merge_list=[lake_iwG_arm_SJ_lyr,lake_niwG_arm_SJ_lyr,lake_arm_lyr,lake_waterway_iwG_SJ_lyr,lake_waterway_niwG_lyr,lake_niwR_lyr,lake_big_lake_lyr]
arcpy.Merge_management(merge_list,temp_file,fieldmappings)

<Result 'J:\\lakemapping\\postprocess\\v4_2403019\\4_polygon_merge.gdb\\lakes_as_4_merge_biglake_temp'>

In [20]:
arcpy.Dissolve_management(temp_file,lake_merge_bl,multi_part="SINGLE_PART")
arcpy.AddField_management(lake_merge_bl, 'area', "DOUBLE")
arcpy.CalculateField_management(lake_merge_bl, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

<Result 'J:\\lakemapping\\postprocess\\v4_2403019\\4_polygon_merge.gdb\\lakes_as_4_merge_biglake'>

In [24]:
target_region_list=[]
origin_region_list_list=[]

In [25]:
target_region = 'as_2'
origin_region_list=['as_1','as_3','as_8']#
target_region_list.append(target_region)
origin_region_list_list.append(origin_region_list)

In [26]:
target_region = 'as_4'
origin_region_list=['as_1','as_2','as_3','as_6','as_7','as_9']
target_region_list.append(target_region)
origin_region_list_list.append(origin_region_list)

In [27]:
target_region = 'as_5'
origin_region_list=['as_3']
target_region_list.append(target_region)
origin_region_list_list.append(origin_region_list)

In [28]:
target_region = 'as_6'
origin_region_list=['as_3','as_5','as_7']
target_region_list.append(target_region)
origin_region_list_list.append(origin_region_list)

In [29]:
target_region = 'as_9'
origin_region_list=['as_7','as_1','eu','af']
target_region_list.append(target_region)
origin_region_list_list.append(origin_region_list)

In [31]:
for i in range(0,5):
    target_region=target_region_list[i]
    origin_region_list=origin_region_list_list[i]
    print(target_region)
    print(origin_region_list)
    start = time.time()
    target_region_file =os.path.join(merge_gdb,'lakes_{}_merge_biglake' .format(target_region))
    target_region_layer='target_region_layer'
    arcpy.MakeFeatureLayer_management(target_region_file, target_region_layer)

    for origin_region in origin_region_list:
        print('select polygons in region:'+ origin_region)
        origin_region_file=os.path.join(merge_gdb,'lakes_{}_merge_biglake' .format(origin_region))
        origin_region_layer='origin_region_layer'
        arcpy.MakeFeatureLayer_management(origin_region_file,origin_region_layer)
        arcpy.SelectLayerByLocation_management(origin_region_layer, 'INTERSECT',target_region_layer )#select lake interect with origin polygon
        output_file='lakes_{}_interect_with_{}' .format(origin_region,target_region)
        arcpy.CopyFeatures_management(origin_region_layer,output_file)
        arcpy.DeleteFeatures_management(origin_region_layer)
    end = time.time()
    print (f'time: {(end-start)/60} minutes')

as_2
['as_1', 'as_3', 'as_8']
select polygons in region:as_1
select polygons in region:as_3
select polygons in region:as_8
time: 0.1469874660174052 minutes
as_4
['as_1', 'as_2', 'as_3', 'as_6', 'as_7', 'as_9']
select polygons in region:as_1
select polygons in region:as_2
select polygons in region:as_3
select polygons in region:as_6
select polygons in region:as_7
select polygons in region:as_9
time: 0.21267534891764323 minutes
as_5
['as_3']
select polygons in region:as_3
time: 0.03888015747070313 minutes
as_6
['as_3', 'as_5', 'as_7']
select polygons in region:as_3
select polygons in region:as_5
select polygons in region:as_7
time: 0.16703265110651652 minutes
as_9
['as_7', 'as_1', 'eu', 'af']
select polygons in region:as_7
select polygons in region:as_1
select polygons in region:eu
select polygons in region:af


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset J:\lakemapping\postprocess\v4_2403019\4_polygon_merge.gdb\lakes_af_merge_biglake does not exist or is not supported
WARNING 000725: Output Layer: Dataset origin_region_layer already exists.
Failed to execute (MakeFeatureLayer).


In [ ]:
target_region='as_9'
origin_region_list=['as_7','eu']

In [38]:
target_region='as_6'
origin_region_list=['as_7']

In [40]:
target_region='as_2'
origin_region_list=['as_3']

In [41]:
print(target_region)
print(origin_region_list)
lak_shp = merge_gdb + '/lakes_{}_merge_biglake_new_temp'.format(target_region)
merge_shp_list =['lakes_{}_merge_biglake' .format(target_region)]
for origin_region in origin_region_list:
    merge_shp_list.append('lakes_{}_interect_with_{}' .format(origin_region,target_region))
arcpy.Merge_management(merge_shp_list, lak_shp) 
lak_dissolved = merge_gdb + '/lakes_{}_merge_biglake_new'.format(target_region) #dissolve geometry
start = time.time()
lak_shp_lyr = 'lak_shp_lyr'
arcpy.MakeFeatureLayer_management(lak_shp, lak_shp_lyr)
print('start Dissolve')
arcpy.Dissolve_management(lak_shp_lyr, lak_dissolved, multi_part="SINGLE_PART")#dissolve copylake
lak_dissolved_Layer='lak_dissolved_lyr'
arcpy.MakeFeatureLayer_management(lak_dissolved, lak_dissolved_Layer)
print('finish dissolve')
arcpy.Delete_management(lak_shp)
end = time.time()
print (f'time: {(end-start)/60} minutes')

as_2
['as_3']
start Dissolve
finish dissolve
time: 2.4354761322339376 minutes
